In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
import PyPDF2

In [67]:
!pip install huggingface_hub
!pip install transformers
!pip install accelerate
!pip install bitsandbytes

In [4]:
KEY=os.getenv("KEY")

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']=KEY

In [72]:
# RESPONSE_JSON = {
#     "1": {
#         "mcq": "multiple choice question",
#         "options": {
#             "a": "choice here",
#             "b": "choice here",
#             "c": "choice here",
#             "d": "choice here",
#         },
#         "correct": "correct answer",
#     },
#     "2": {
#         "mcq": "multiple choice question",
#         "options": {
#             "a": "choice here",
#             "b": "choice here",
#             "c": "choice here",
#             "d": "choice here",
#         },
#         "correct": "correct answer",
#     },
#     "3": {
#         "mcq": "multiple choice question",
#         "options": {
#             "a": "choice here",
#             "b": "choice here",
#             "c": "choice here",
#             "d": "choice here",
#         },
#         "correct": "correct answer",
#     },
# }

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone"],
    template=TEMPLATE
    )

In [10]:
llm=HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha", model_kwargs={'temperature':0.005})

/var/folders/vw/24j_z4k52kjgr5sq77sd2_h40000gn/T/ipykernel_7987/1280314016.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  llm=HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha", model_kwargs={'temperature':0.005})
/Users/soham/Documents/GitHub/MCQ-Generator-using-LLM-and-Langchan/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
quiz_chain = LLMChain(llm=llm,prompt = quiz_generation_prompt, output_key="quiz", verbose=True)

/var/folders/vw/24j_z4k52kjgr5sq77sd2_h40000gn/T/ipykernel_7987/3646152006.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain = LLMChain(llm=llm,prompt = quiz_generation_prompt, output_key="quiz", verbose=True)


In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [14]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone"],output_variables=["quiz", "review"], verbose=True,)

In [17]:
file_path=r"/Users/soham/Documents/GitHub/MCQ-Generator-using-LLM-and-Langchan/experiment/data.txt"

In [18]:
file_path

'/Users/soham/Documents/GitHub/MCQ-Generator-using-LLM-and-Langchan/experiment/data.txt'

In [19]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [20]:
print(TEXT)

Quantum mechanics is the field of physics that explains how extremely small objects simultaneously have the characteristics of both particles (tiny pieces of matter) and waves (a disturbance or variation that transfers energy). Physicists call this the “wave-particle duality.”

The particle portion of the wave-particle duality involves how objects can be described as “quanta.” A quanta is the smallest discrete unit (such as a particle) of a natural phenomenon in a system where the units are in a bound state. For example, a quanta of electromagnetic radiation, or light, is a photon. A bound state is one where the particles are trapped. One example of a bound state is the electrons, neutrons, and protons that are in an atom.

To be “quantized” means the particles in a bound state can only have discrete values for properties such as energy or momentum. For example, an electron in an atom can only have very specific energy levels. This is different from our world of macroscopic particles, 

In [86]:
# # Serialize the Python dictionary into a JSON-formatted string
# json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5 
SUBJECT="Quantum"
TONE="Scietific"


In [22]:
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
    
    })

/var/folders/vw/24j_z4k52kjgr5sq77sd2_h40000gn/T/ipykernel_7987/3683820470.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Quantum mechanics is the field of physics that explains how extremely small objects simultaneously have the characteristics of both particles (tiny pieces of matter) and waves (a disturbance or variation that transfers energy). Physicists call this the “wave-particle duality.”

The particle portion of the wave-particle duality involves how objects can be described as “quanta.” A quanta is the smallest discrete unit (such as a particle) of a natural phenomenon in a system where the units are in a bound state. For example, a quanta of electromagnetic radiation, or light, is a photon. A bound state is one where the particles are trapped. One example of a bound state is the electrons, neutrons, and protons that are in an atom.

To be “quantized” means the particles in a bound state can only have discrete values for properties such as energy or momentum. For example, an electron in an

In [23]:
print(response)

{'text': 'Quantum mechanics is the field of physics that explains how extremely small objects simultaneously have the characteristics of both particles (tiny pieces of matter) and waves (a disturbance or variation that transfers energy). Physicists call this the “wave-particle duality.”\n\nThe particle portion of the wave-particle duality involves how objects can be described as “quanta.” A quanta is the smallest discrete unit (such as a particle) of a natural phenomenon in a system where the units are in a bound state. For example, a quanta of electromagnetic radiation, or light, is a photon. A bound state is one where the particles are trapped. One example of a bound state is the electrons, neutrons, and protons that are in an atom.\n\nTo be “quantized” means the particles in a bound state can only have discrete values for properties such as energy or momentum. For example, an electron in an atom can only have very specific energy levels. This is different from our world of macroscop

In [24]:
print(response["quiz"])


Text:Quantum mechanics is the field of physics that explains how extremely small objects simultaneously have the characteristics of both particles (tiny pieces of matter) and waves (a disturbance or variation that transfers energy). Physicists call this the “wave-particle duality.”

The particle portion of the wave-particle duality involves how objects can be described as “quanta.” A quanta is the smallest discrete unit (such as a particle) of a natural phenomenon in a system where the units are in a bound state. For example, a quanta of electromagnetic radiation, or light, is a photon. A bound state is one where the particles are trapped. One example of a bound state is the electrons, neutrons, and protons that are in an atom.

To be “quantized” means the particles in a bound state can only have discrete values for properties such as energy or momentum. For example, an electron in an atom can only have very specific energy levels. This is different from our world of macroscopic parti

In [58]:
print(llm.invoke("Create MCQ on topic Quatum Mechanics"))

Create MCQ on topic Quatum Mechanics.
Question: What is the wave-particle duality?
A) The idea that particles can behave like waves
B) The idea that waves can behave like particles
C) The idea that particles and waves are completely separate concepts
D) The idea that particles and waves are the same thing

Question: What is the uncertainty principle in quantum mechanics?
A) The principle that the more precisely the position of a particle is known, the less precisely its momentum can be known


In [25]:
# Assuming you have a language model (llm) set up
print(llm.invoke("""
Text: Quantum mechanics is the field of physics that explains how extremely small objects simultaneously have the characteristics of both particles (tiny pieces of matter) and waves (a disturbance or variation that transfers energy). Physicists call this the "wave-particle duality."

The particle portion of the wave-particle duality involves how objects can be described as "quanta." A quanta is the smallest discrete unit (such as a particle) of a natural phenomenon in a system where the units are in a bound state. For example, a quanta of electromagnetic radiation, or light, is a photon. A bound state is one where the particles are trapped. One example of a bound state is the electrons, neutrons, and protons that are in an atom.

To be "quantized" means the particles in a bound state can only have discrete values for properties such as energy or momentum. For example, an electron in an atom can only have very specific energy levels. This is different from our world of macroscopic particles, where these properties can be any value in a range. A baseball can have essentially any energy as it is thrown, travels through the air, gradually slows down, then stops.

At the same time, tiny quantized particles such as electrons can also be described as waves. Like a wave in the ocean in our macroscopic world – the world we can see with our eyes – waves in the quantum world are constantly shifting. In quantum mechanics, scientists talk about a particle's "wave function." This is a mathematical representation used to describe the probability that a particle exists at a certain location at a certain time with a certain momentum.

You are an expert MCQ maker. Given the above text, it is your job to create a quiz of 5 multiple choice questions for Quantum students in a scientific tone. Make sure the questions are not repeated and check that all the questions conform to the text.

Make sure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make 5 MCQs:

  """     
))


Text: Quantum mechanics is the field of physics that explains how extremely small objects simultaneously have the characteristics of both particles (tiny pieces of matter) and waves (a disturbance or variation that transfers energy). Physicists call this the "wave-particle duality."

The particle portion of the wave-particle duality involves how objects can be described as "quanta." A quanta is the smallest discrete unit (such as a particle) of a natural phenomenon in a system where the units are in a bound state. For example, a quanta of electromagnetic radiation, or light, is a photon. A bound state is one where the particles are trapped. One example of a bound state is the electrons, neutrons, and protons that are in an atom.

To be "quantized" means the particles in a bound state can only have discrete values for properties such as energy or momentum. For example, an electron in an atom can only have very specific energy levels. This is different from our world of macroscopic part

# WE COULD HAVE RECIEVED EVERYTHING AS PLANNED IF THE LLM WAS MAYBE A MODEL LIKE GPT 3.5 or similar but I dont have any OPENAI credits :(